In [ ]:
# Dependências

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
# Arquivos de Entrada
image_path = ""
file_path = "/filter.txt"

In [ ]:
# Funções para abrir, exibir e salvar imagens
# Funções gerais/essênciais

def open_image(file_path):
  image = Image.open(file_path)
  return image

def display_image(image):
  image.show()

def display_image_np(image):
  plt.imshow(image)

def display_image_from_array(array, mode):
  image = Image.fromarray(array, mode)
  image.show()

def save_image(image, file_path):
  image.save(file_path)

In [ ]:
# Conversão de espaços de cor (RGB e HSB)

def rgb2hsv(image_rgb):
  # Converte a imagem RGB em um array NumPy
  image_array = np.array(image_rgb)
  
  # Normalização dos valores de RGB
  image_array = image_array / 255.0
  
  # Extrai os canais R, G e B
  r, g, b = image_array[:, :, 0], image_array[:, :, 1], image_array[:, :, 2]
  
  # Calcula o valor máximo e mínimo
  valor_max = np.amax(image_array, axis=2)
  valor_min = np.amin(image_array, axis=2)
  
  # Calcula os índices dos valores máximos e mínimos
  valormax_indice = np.argmax(image_array, axis=2)
  valormin_indice = np.argmin(image_array, axis=2)
  
  diferenca = valor_max - valor_min
  
  #cria uma matriz numpy igual a matriz de entrada, valores com tipo float
  hsv = np.zeros(image_array.shape, dtype='float')
  
  #INDICE 0(RED), 1(GREEN) , 2(BLUE) | utilizar np.spacing para não fazer divisões por zero
  # 0(Hue), 
  hsv[(valormax_indice == 0) & (image_array[..., 1] >= image_array[..., 2]), 0] = (((image_array[...,1] - image_array[..., 2]) * 60.0 / (valor_max - valor_min + np.spacing(1))) + 0.0)[(valormax_indice == 0) & (image_array[..., 1] >= image_array[..., 2])]
  hsv[(valormax_indice == 0) & (image_array[..., 1] < image_array[..., 2]), 0] = (((image_array[...,1] - image_array[..., 2]) * 60.0 / (valor_max - valor_min + np.spacing(1))) + 360.0)[(valormax_indice == 0) & (image_array[..., 1] < image_array[..., 2])]
  hsv[valormax_indice == 1, 0] = (((image_array[...,2] - image_array[..., 0]) * 60.0 / (valor_max - valor_min + np.spacing(1))) + 120.0)[valormax_indice == 1]
  hsv[valormax_indice == 2, 0] = (((image_array[...,0] - image_array[..., 1]) * 60.0 / (valor_max - valor_min + np.spacing(1))) + 240.0)[valormax_indice == 2]
  #1(Saturação), 
  hsv[valor_max == 0, 1] = np.zeros(hsv[valor_max == 0 , 1].shape)
  hsv[valor_max != 0, 1] = (1 - valor_min / (valor_max + np.spacing(1)))[valor_max != 0]
  #2 (Brightness)
  hsv[..., 2] = valor_max
  
  #retorna uma matriz tipo numpy
  return hsv

#Para visualizar a matriz numpy como 0-360(grau),0-100(%),0-100(%)
def int_per_transform(np_array):
  matriz_int_perc = np.round(np_array * 100)
  matriz_int_perc[...,0] = np_array[...,0].astype(int)
  
  return matriz_int_perc

def hsv2rgb(hsv):
  
  #np.floor faz um arredondamento para baixo do valores encontrados.
  #hi é a condição que determinará quais valores serão atribuidos a RGB
  hi = np.floor(hsv[..., 0] / 60.0) % 6
  #transforma os valores para 8 bits (0-255)
  hi = hi.astype('uint8')
  # brightness / value / luminância
  v = hsv[..., 2].astype('float')
  # hue / matiz
  f = (hsv[..., 0] / 60.0) - np.floor(hsv[..., 0] / 60.0)
  # saturação
  p = v * (1.0 - hsv[..., 1])
  q = v * (1.0 - (f * hsv[..., 1]))
  t = v * (1.0 - ((1.0 - f) * hsv[..., 1]))
  
  #Cria uma matriz de zeros com as mesmas medidas da matriz de entrada
  rgb = np.zeros(hsv.shape)
  # Caso os valores de hi seja ___ np.dstack irá atribuir para a matriz de rgb determinados valores
  # dstack -> valores prontos para os pixels da matriz
  rgb[hi == 0, :] = np.dstack((v, t, p))[hi == 0, :]
  rgb[hi == 1, :] = np.dstack((q, v, p))[hi == 1, :]
  rgb[hi == 2, :] = np.dstack((p, v, t))[hi == 2, :]
  rgb[hi == 3, :] = np.dstack((p, q, v))[hi == 3, :]
  rgb[hi == 4, :] = np.dstack((t, p, v))[hi == 4, :]
  rgb[hi == 5, :] = np.dstack((v, p, q))[hi == 5, :]

  return rgb

rgb2hsv.__doc__ = "Converte do espaço RGB para HSB, retorna uma matriz numpy"
int_per_transform.__doc__ = "Converte do espaço HSB para RGB, retorna uma matriz numpy"
hsv2rgb.__doc__ = "Converte do espaço HSB para RGB, retorna uma matriz numpy"

In [ ]:
# Alteraçaõ de Matiz e Saturação
def change_hue_and_saturation(image, delta_hue, delta_saturation):
  # Converte a imagem RGB em HSV
  hsv = rgb2hsv(image)
  
  # Garante que a delta_hue esteja dentro do intervalo de -360 a 360
  delta_hue = np.clip(delta_hue, -360, 360)

  # Garante que a delta_saturation esteja dentro do intervalo de -1 a 1
  delta_saturation = np.clip(delta_saturation, -1, 1)

  # Aplique a alteração de matiz para todos os pixels
  hsv[..., 0] += delta_hue

  # Verifique e ajuste os valores que ultrapassam os limites (relogio)
  hsv[..., 0] = np.where(hsv[..., 0] > 360, hsv[..., 0] - 360, hsv[..., 0]) # subtrai
  hsv[..., 0] = np.where(hsv[..., 0] < 0, hsv[..., 0] + 360, hsv[..., 0]) # adiciona
  
  # Modifica a saturação (raio)
  hsv[..., 1] = np.clip(hsv[..., 1] + delta_saturation, 0, 1)
  
  # Converte HSV de volta para RGB
  modified_rgb = hsv2rgb(hsv)
  
  return modified_rgb

change_hue_and_saturation.__doc__ = "Recebe uma imagem RGB, converte para o espaço HSB, altera a matiz e saturação e retorna a imagem RGB transformada."


In [ ]:
# Negativo 
# criar negativo do hsv(apenas o valor V) com conversão para rgb
def negativo_hsv(image):
  
  #transforma a imagem de entrada para hsv
  image_hsv = rgb2hsv(image)
  
  #faz o negativo do canal 2 (V), np.clip para que os valores estejam entre 0 e 1 
  image_hsv[:,:,2] = np.clip(1 - image_hsv[:,:,2], 0, 1)
  
  #transforma para rgb novamente para poder visualizar o resultado
  image_negative = hsv2rgb(image_hsv)
  
  return image_negative

# criar negativo rgb (banda a banda) 
def negativo_rgb(image):
  
  #Como não é necessário usar nenhuma das funções de conversão , faz-se necessário converter a imagem para array numpy
  image_np = np.array(image)
  
  #atribui para r, g e b os seus respectivos canais de cores
  r, g, b = image_np[:,:,0], image_np[:,:,1], image_np[:,:,2]
  
  #Cálculo do negativo de cada banda, np.clip para que os valores não ultrapassem 0-255
  r_negativo = np.clip(255 - r, 0, 255)
  g_negativo = np.clip(255 - g, 0, 255)
  b_negativo = np.clip(255 - b, 0, 255)
  
  #np.stack para colocar os pixels (RGB)
  image_negative = np.stack((r_negativo, g_negativo, b_negativo), axis = -1)
  
  return image_negative

In [ ]:
def ler_arquivo(parametro):
  with open(parametro, 'r') as f:
    linhas = f.readlines()
    m, n = map(int, linhas[0].split())  # Extrair dimensões m e n do filtro
    stride = int(linhas[1])             # Extrair o valor do stride
    filtro = linhas[2].strip()          # Extrair o nome do filtro (Box ou Sobel)
  return m, n, stride, filtro

In [ ]:
# Correlação
# Função que realiza a operação de correlação entre uma imagem e um filtro
def correlacao(imagem, filtro, stride):
  altura, largura, _ = imagem.shape            # Obter dimensões da imagem
  f_altura, f_largura = filtro.shape           # Obter dimensões do filtro
  resultado = np.zeros_like(imagem)            # Inicializar imagem de resultado com zeros

  # Loop através da imagem com o passo definido pelo stride

  # Percorrendo a imagem verticalmente
  for y in range(0, altura - f_altura + 1, stride):
    # Percorrendo a imagem horizontalmente
    for x in range(0, largura - f_largura + 1, stride):
      # Garantindo que a correlação seja aplicada de forma independente
      for c in range(3):                                # Itera pelos canais R, G e B da imagem
        sec = imagem[y:y+f_altura, x:x+f_largura, c]  # Extrai segmento da imagem
        resultado[y, x, c] = np.abs(np.sum(sec * filtro))     # Realizar correlação e armazenar no resultado

  return resultado

In [ ]:
# Expansão do histograma para realçar os resultados
def expansao_histograma(imagem):
  minimo = np.min(imagem)
  maximo = np.max(imagem)
  return  np.abs(((imagem - minimo) / (maximo - minimo)) * 255.0)

In [ ]:
# main()

# Função principal
def main():

  # abrindo imagem rgb
  image_rgb = open_image(image_path)
  display_image(image_rgb)

  # convertendo para hsb e exibindo 
  image_hsb_np = rgb2hsv(image_rgb)
  display_image_from_array(int_per_transform(image_hsb_np), 'HSV')
  image_rgb_back = hsv2rgb(image_hsb_np) 

  # exibindo a imagem de volta para rgb
  display_image_from_array((image_rgb_back * 255).astype(np.uint8), 'RGB')

  # 2. Alterar matiz e saturação no HSB e converter para RGB
  modified_image = change_hue_and_saturation(image_rgb, 360, 0)
  display_image_np(modified_image)

  # 3. Criar o negativo da imagem

  hsv_negativo = negativo_hsv(image_rgb)
  rgb_negativo = negativo_rgb(image_rgb)
  
  plt.figure(figsize=(10,5))
  plt.subplot(1,2,1)
  plt.imshow(hsv_negativo)
  plt.title('hsv_negativo')
  plt.subplot(1,2,2)
  plt.imshow(rgb_negativo)
  plt.title('rgb_negativo')
  plt.tight_layout()
  plt.show()

  # 4. Aplicar correlação m x n com stride sobre R, G e B
  # Carregar imagem e convertê-la para um array
  imagem_box = Image.open(image_path)
  imagem_sobel = Image.open(shapes)
  imagem_box = np.array(imagem_box)
  imagem_sobel = np.array(imagem_sobel)

  # Ler configurações do arquivo
  m, n, stride, filtro_nome = ler_arquivo(parametro)

  # Aplicar filtros Box ou Sobel 
  if filtro_nome.lower() == 'box':
    # Medir o tempo de execução para Box 15x1 seguido de Box 1x15
    inicio = time.time()
    resultado1 = correlacao(imagem_box, box(m, 1), stride)
    resultado1 = correlacao(resultado1, box(1, n), stride)
    fim1 = time.time() - inicio

    # Medir o tempo de execução para Box 15x15
    inicio = time.time()
    resultado2 = correlacao(imagem_box, box(m, n), stride)
    fim2 = time.time() - inicio

    # Mostrando as imagens apos os filtros aplicados
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(resultado1)
    plt.title('1x15 15x1')
    plt.subplot(1,2,2)
    plt.imshow(resultado2)
    plt.title('15x15')
    plt.tight_layout()
    plt.show()

    # Mostrar os tempos medidos
    print("Tempo Box 15x1(Box 1x15):", fim1)
    print("Tempo Box 15x15:", fim2)

  # Aplicação do filtro Sobel
  elif filtro_nome.lower() == 'sobel':
    # Aplicação do filtro Sobel na horizontal e vertical
    resultado_horizontal = correlacao(imagem_sobel, sobel_horizontal(), stride)
    resultado_vertical = correlacao(imagem_sobel, sobel_vertical(), stride)
    
    
    # Combinar resultados para obter detecção completa de bordas
    sobel_result = np.abs((resultado_horizontal) + (resultado_vertical))

    # Correção do intervalo da imagem
    if sobel_result.dtype == np.float64 or sobel_result.dtype == np.float32:
      sobel_result = np.clip(sobel_result, 0.0, 1.0)
    else:
      sobel_result = np.clip(sobel_result, 0, 255).astype(np.uint8)

    plt.imshow(sobel_result, cmap='gray')
    plt.title('resultado do filtro sobel')
    plt.colorbar()
    plt.show()

    sobel_result = expansao_histograma(sobel_result)         # Expande o histograma

    # Correção do intervalo da imagem
    if sobel_result.dtype == np.float64 or sobel_result.dtype == np.float32:
      sobel_result = np.clip(sobel_result, 0.0, 1.0)
    else:
      sobel_result = np.clip(sobel_result, 0, 255).astype(np.uint8)

    plt.imshow(sobel_result, cmap='gray')
    plt.title(' filtro sobel com expansão do histograma')
    plt.colorbar()
    plt.show()
      

  else:
    print("Erro: O nome do filtro está errado!")  

if __name__ == "__main__":
  main()
